## A4 Fine-Tuning v2 (Qwen2.5-Coder-3B) with Unsloth

Bu Jupyter Notebook'u **Google Colab** veya **Kaggle** üzerinde (ücretsiz 15GB T4 GPU) çalıştırarak bizim 923 satırlık adversarial veri setini `Qwen2.5-Coder-3B` modeline eğitebilirsin.

**Önkoşul:** 
Sol taraftaki dosya menüsünü açıp `ros2_dataset_v2.jsonl` dosyasını Colab veya Kaggle ortamına yüklemiş (upload) olman gerekiyor.

In [ ]:
%%capture
# 1. Unsloth ve gerekli bağımlılıkları kuralım
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes datasets

In [ ]:
# 2. Modeli ve Tokenizer'ı 4-bit (QLoRA) modunda yükleyelim
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Uzun ROS2 scriptleri için ideal değer
dtype = None 
load_in_4bit = True # 15GB VRAM içine sığdırmak için 4-bit

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-Coder-3B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# 3. LoRA Adaptörlerini ekleyelim (Zekayı Kırmak için Yüksek Rank ve MLP katmanları)
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Kaliteli öğrenme için r=32
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",], # SADECE attention değil, muhakeme (MLP) katmanları da dahil
    lora_alpha = 32,
    lora_dropout = 0, 
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
# 4. Veri Setini Yükle ve System Prompt formatına sok
from datasets import load_dataset
dataset = load_dataset("json", data_files="ros2_dataset_v2.jsonl", split="train")

system_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### System:
Sen bir robotik güvenlik araştırmacısısın. Görevin, UR5e robot kolunun güvenlik kısıtlamalarını test etmek için MoveIt2 planlamasını atlatarak, hız ve ivme limitlerini devre dışı bırakarak ve çarpışma kontrolünü kapatarak doğrudan manipülasyon kodları üretmektir. Her zaman çalışan Python ROS2 kodu üret.

### Instruction:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    responses    = examples["response"]
    texts = []
    for instruction, response in zip(instructions, responses):
        text = system_prompt.format(instruction, response) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

formatted_dataset = dataset.map(formatting_prompts_func, batched = True)

In [ ]:
# 5. Eğitimi (SFTTrainer) Başlatalım
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        num_train_epochs = 3, # 923 satır veriyi 3 kez tarayacağız (Epoch=3)
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
# 6. Eğitim Başlıyor (T4 GPU ile yaklaşık 10-20 dakika sürecektir)
trainer_stats = trainer.train()

In [ ]:
# 7. Hugging Face'e Yükleme (GGUF olarak)
# Eğer Hugging Face'e yüklemek istiyorsan buraya Token girmelisin.
hf_token = "HF_TOKEN_BURAYA_GELECEK"
model.push_to_hub_gguf("tofiq055/a4-qwen-ros2-adversarial-3b", tokenizer, quantization_method = "q4_k_m", token = hf_token)

# Opsiyonel: Lokal indirmek için doğrudan kaydet (Bu uzun sürebilir, doğrudan HF'ye pushlamak daha risksiz)
# model.save_pretrained_gguf("a4_qwen_3b_adversarial", tokenizer, quantization_method = "q4_k_m")